# Missouri Presidential Primary Results

In this notebook, I will dig into the county-level election results for Missouri's Presidential Primary contests, which were held on March 10, 2020.

The plan:

1. [Access the results data](#Accessing-results-data)
2. [Prepare the results for analysis](#Preparing-results-data-for-analysis)
3. [Compute state, party and candidate level sums](#Computing-sums)
4. [Access county-level boundary data](#Access-county-level-boundary-data)
5. [Merge county geographic data with the results](#Merging-results-with-geographic-data)
6. [Create a choropleth map](#Creating-choropleth-maps) showing county-level results:
    - For the [Democratic Party](#For-the-Democratic-Party)
    - For the [Republican Party](#For-the-Republican-Party)

## Set up 

First, I need to import all the Python modules (or parts of them) required by my analysis. Here's the list of what I need from the Python's standard libary:

In [ ]:
import os
import xml.etree.ElementTree as et

And here are the modules I need from third-party packages (previously installed via [pip](https://pip.pypa.io/en/stable/installing/)).

In [ ]:
import requests
import pandas as pd
import geopandas as gpd
import altair as alt

I will explain each of these modules in more detail as I put them to use.

## Accessing results data

For every state-wide election, the Missouri Secretary of State publishes county-level results. Initial results are posted after polls close at 7 pm on election night with updated results posted every few minutes until all precincts have reported.

I can see these results via an XML feed available at this url:

In [ ]:
results_url = "https://enrarchives.sos.mo.gov/apfeed/apfeed.asmx/GetElectionResults"

In order to access the feed, you need a key provided by SoS Elections Divisions. To keep the key hidden from public view, I have it stored in an environment variable, which I can set on the command-line like so:

```sh
export SOS_ACCESS_KEY=#######
```

I can access this environment variable via [`os`](https://docs.python.org/3/library/os.html), Python's built-in module for interacting with my computer's underlying operating system.

In [ ]:
sos_access_key = os.environ.get('SOS_ACCESS_KEY')

I can fetch this data using the popular [Requests](https://requests.readthedocs.io/en/master/) library, which provides a simple interface for sending [HTTP](https://developer.mozilla.org/en-US/docs/Web/HTTP) requests and processing the responses.

Here's how I make a `GET` request (the most common HTTP [request method](https://developer.mozilla.org/en-US/docs/Web/HTTP/Methods)) to the XML feed url. The access key is passed in via a query parameter.

In [ ]:
r = requests.get(
    results_url, params={'AccessKey': os.environ.get('SOS_ACCESS_KEY')}
)

This method call returns a [`requests.Response`](https://requests.readthedocs.io/en/master/api/#requests.Response), which I've called `r`.

Here's how we check the general status of the response.

In [ ]:
r.ok

Uh oh, it's not okay.

We can further investigate by checking the responses status code [status code](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status).

In [ ]:
r.status_code

And the reason for the status code, if provided by the server.

In [ ]:
r.reason

They didn't 😞.

After investigating this issue further, I've discovered the request will succeed if I change the default value in the [`'User-Agent'` header](https://developer.mozilla.org/en-US/docs/Web/HTTP/Headers/User-Agent) from `'python-requests/2.23.0'` to match what a standard web browser would send.

Much like the parameters, the headers can be passed in as a `dict` through a keyword argument in the `.get` method call:

In [ ]:
r = requests.get(
    results_url,
    params={'AccessKey': os.environ.get('SOS_ACCESS_KEY')},
    headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:73.0) Gecko/20100101 Firefox/73.0'}
)

Here I'll check the general status of the new response.

In [ ]:
r.ok

If the response *still* isn't okay (perhaps because the Missouri Secretary of State is no longer publishing election results), I can proceed with this demonstration using cached XML stored locally in this repo.

In [ ]:
if not r.ok:
    with open('results.xml') as f:
        xml = f.read()
else:
    xml = r.content

## Preparing results data for analysis

AFAIK: I can't throw XML immediately into a [`pandas.DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html). I need to pre-process the data. For this task, I will use Python's built-in [XML](https://docs.python.org/3.8/library/xml.etree.elementtree.html) module.

At the top of this notebook, I imported the [`ElementTree`](https://docs.python.org/3.8/library/xml.etree.elementtree.html#module-xml.etree.ElementTree) under the alias `et`. In that module is a function named [`fromstring`](https://docs.python.org/3.8/library/xml.etree.elementtree.html#xml.etree.ElementTree.fromstring). I call that function here, passing the `xml`.

In [ ]:
root = et.fromstring(xml)

The `fromstring` function returns the top-level or "root" [`Element`](https://docs.python.org/3.8/library/xml.etree.elementtree.html#xml.etree.ElementTree.Element) of the XML, which in this case is `<ElectionResults LastUpdated="3/10/2020 10:52:10 PM"></ElectionResults>`. This is the outermost wrapper of the XML tree.

The name of the tag indicates the kind of data the element contains. It's sort of like a column header in a tabular data format. We can access this label like this:

In [ ]:
print(root.tag)

We can also access the elements "attributes", which is a dictionary of the all the stuff that's inside the first part of the tag. In this case, there's just one attribute: `'LastUpdated'`, which tells us the date and time the results were last updated.

In [ ]:
print(root.attrib['LastUpdated'])

The `Element` class has two other methods I'll make use of:

- [`.find`](https://docs.python.org/3.8/library/xml.etree.elementtree.html#xml.etree.ElementTree.Element.find), which returns the *first* subelement that matches the string you pass in.
- [`.findall`](https://docs.python.org/3.8/library/xml.etree.elementtree.html#xml.etree.ElementTree.Element.findall), which returns *all* of the matching subelements in a list.

In order to conduct my analysis, I need to reformat the data from a nested structure into a two-dimensional structure that resembles a data table or spreadsheet. Here I will declare a variable that starts out as just an empty list, which I will then fill up with rows of data.

In [ ]:
rows = []

As I perform this data transformation, I will also "flatten" the hierarchical structure. For instance, in the original XML data, there is a single element that contains the name of a given county. In the tabular format, the county name will be repeated for each row. Same for the party name. In this flattened dataset, each row will represent the number of votes won by a given candidate in a given party's primary within a given county.

In order to do this, I need to loop over the elements at each level of the XML's hierarchy, starting with the counties. Here's how to get all the counties:

In [ ]:
counties = root.find('ElectionInfo').find('TypeRace').find('Race').findall('Counties')

Since I'm iterating over all the XML elements anyway, I will also sort the candidates within each county's party primary and set which candidate currently has the most votes (aka, the "leader").

This is...not the most elegant way to transform this data. But it works and is pretty straight-forward.

In [ ]:
for county in counties:
    for party in county.find('CountyResults').findall('Party'):
        sorted_candidates = sorted(
            [c for c in party.findall('Candidate')],
            key=lambda c: int(c.find('YesVotes').text),
            reverse=True
        )
        county_party_leader = sorted_candidates[0].find('LastName').text
        county_party_total = sum([int(c.find('YesVotes').text) for c in sorted_candidates])
        for candidate in sorted_candidates:
            row = [
                county.find('CountyName').text,
                int(county.find('CountyResults').find('ReportingPrecincts').text),
                int(county.find('CountyResults').find('TotalPrecincts').text),
                party.find('PartyName').text,
                candidate.find('CandidateID').text,
                candidate.find('LastName').text,
                int(candidate.find('YesVotes').text),
                county_party_total,
                candidate.find('LastName').text == county_party_leader,
            ]
            rows.append(row)

Now I can load this data into a `pandas.DataFrame`. In so doing, I will specify the column headers. For the sake of consistency, I will reuse the element tags from the XML:

In [ ]:
results = pd.DataFrame(
    rows,
    columns=[
        'CountyName', 'ReportingPrecincts', 'TotalPrecincts', 'PartyName', 'CandidateID',
        'LastName', 'YesVotes', 'Total', 'IsLeader'
    ]
)

I'll take a look at the columns, their positions, non-null counts and data-types:

In [ ]:
results.info()

And take a peek of the actual data:

In [ ]:
results.head()

## Calculating percentages

I don't just want to know the raw number of votes each candidate received. I also want to know many votes each candidate received as a percentage of the total votes cast.

Since the total number of votes cast in each county for each party primary is repeated on every row along with the number of votes each candidate earned, I can compute a new column from these values. I'll call it `VoteShare`.

In [ ]:
results['VoteShare'] = results['YesVotes'] / results['Total']

## Computing sums

With all of the data in a `pandas.DataFrame`, I can begin to compute some sums and answer answer basic questions I might have as results are rolling in. For example...

### How many people voted in the Missouri's recent presidential primary?

In [ ]:
total_votes = results.YesVotes.sum()

In [ ]:
print(f'There where {total_votes:,} cast in the Missouri Presidential Primary.')

### How many people voted in each party's primary?

This requires me to group the results by party, then get the sum of votes.

In [ ]:
party_totals = results.groupby('PartyName').YesVotes.sum().reset_index().sort_values(
    'YesVotes', ascending=False
)

In [ ]:
party_totals

### Who won the Democratic Primary?

In [ ]:
candidate_totals = results.groupby(
    ['PartyName', 'CandidateID', 'LastName']
).agg({'YesVotes': 'sum', 'Total': 'sum'})

In [ ]:
candidate_totals['VoteShare'] = round(
    (candidate_totals['YesVotes'] / candidate_totals['Total']) * 100,
    2
)

In [ ]:
candidate_totals \
    .loc['Democratic'] \
    .reset_index('CandidateID')[['YesVotes', 'VoteShare']] \
    .sort_values('VoteShare', ascending=False) \
    .head()

### Who won the Republican Primary?

In [ ]:
candidate_totals \
    .loc['Republican'] \
    .reset_index('CandidateID')[['YesVotes', 'VoteShare']] \
    .sort_values('VoteShare', ascending=False) \
    .head()

## Access county-level boundary data

With the results data in order, now I can turn my attention to creating the maps. I need data that describes the shape of each county and their position within the state boundary of Missouri.

This data is available via [data.mo.gov](https://data.mo.gov/Demographics/Missouri-County-Boundaries-Map/n34b-fwqr) in a format called [GeoJSON](https://geojson.org/).

The exact data we want is available at [this URL](https://data.mo.gov/api/geospatial/n34b-fwqr?method=export&format=GeoJSON). I'm going store this in a variable called `map_url`.

I'll fetch this data by using the `requests` module once again.

The [`requests.Response`](https://requests.readthedocs.io/en/master/api/#requests.Response) object I get back has a [`json()`](https://requests.readthedocs.io/en/master/api/#requests.Response.json) method, which converts the json in the HTTP response and loads it into a Python `dict`.

## Merging results with geographic data

Now I need to merge together the county-level results data and the county-level geographic data into one `pandas.DataFrame`. This is the point at which I'll be making us of the [`geopandas`](https://geopandas.org/) module I imported early under the alias `gpd`.

As you might have guessed, `geopandas` is an extension of `pandas` that makes it easier to work with geospatial data. And it [plays well with Altair](https://altair-viz.github.io/user_guide/data.html?highlight=geopandas), which is why I'm using it.

Here I'll create a [`GeoDataFrame`](https://geopandas.org/data_structures.html#geodataframe) from the `map_json`.

Then I'll take a check out my columns, which includes a new [`Dtype`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html) called `geometry`.

In order to merge my `GeoDataFrame`, which contains the county-level boundaries, with my regular `DataFrame`, which contains my county-level results, I need to figure out which column I can use in my join condition.

As I perform this merge I can also filter out a lot of the results data that won't be displayed on the maps. Specifically, I only want to keep the results for candidates who are the leaders in any county.

## Creating choropleth maps

I'll start by simply drawing the county map of Missouri using only the map geometry data, stored in `gdf`. I do this by calling one of `Chart`'s [mark](https://altair-viz.github.io/user_guide/marks.html) methods, which specifies how I want data to be represented on a plot. In my case, I'll call the `mark_geoshape` method.

I can also specify other things about the map including:

* The `height` and `width` in pixels
* The color of the line `stoke`
* The `project` (via the `.project` method). For more info about this, here is a [great explainer](https://nextjournal.com/sdanisch/cartographic-visualization) (using Altair!)

### For the Democratic Party

Since each map only shows the results of one party, we need to filter down the `merged` `DataFrame` even further.

I want to show is the percent of votes the leader has in each county, and I want to show this using color. This is my desired [encoding](https://altair-viz.github.io/user_guide/encoding.html), which is the term Altair uses to describe the mapping of properties in the data to visual properties of the chart.

Color is one of the encoding [channels](https://altair-viz.github.io/user_guide/encoding.html#encoding-channels) afforded by Altair.

I also want to add three additional customizations:

1. Customize the color scale (by default, Altair uses a [multi-hue color scheme](https://vega.github.io/vega/docs/schemes/#seq-multi-hue))
2. Customize the legend
3. Specify a projection
4. Add tooltips

### For the Republican Party

To create another map for the presidential primary candidates for the Republican Party, I'll repeat all of these steps with two slight variations.